In [1]:
import sys
from database_handler import DatabaseHandler

sys.path.append("..")

In [5]:
db = DatabaseHandler(db_path='../data/database/stocks.db')

('AAPL', 'Apple Inc.', 'Information Technology', '2024-05-02', 1, 1)

In [8]:
db.fetch_insider_data("AMZN")

,tic,filling_date,trade_date,owner_name,title,transaction_type,last_price,qty,shares_held,owned,value
0,AMZN,2005-08-03,2005-08-01,Alberg Tom A,Dir,S - Sale,$44.81,"-33,470","195,865",-15%,"-$1,499,791"
1,AMZN,2005-08-03,2005-08-01,Kilar Jason,SVP,S - Sale+OE,$44.75,"-15,000","30,187",-33%,"-$671,181"
2,AMZN,2005-08-03,2005-08-01,Szkutak Thomas J,"SVP, CFO",S - Sale,$44.68,"-30,000","28,015",-52%,"-$1,340,256"
3,AMZN,2005-08-03,2005-08-01,Wilke Jeffrey A,SVP,S - Sale+OE,$45.05,"-33,750","40,149",-46%,"-$1,520,384"
4,AMZN,2005-08-03,2005-08-01,Wilson L Michelle,SVP,S - Sale+OE,$44.77,"-25,000","32,395",-44%,"-$1,119,193"
...,...,...,...,...,...,...,...,...,...,...,...
998,AMZN,2024-05-23,2024-05-21,Zapolsky David,SVP,S - Sale+OE,$181.41,"-9,490","64,210",-13%,"-$1,721,603"
999,AMZN,2024-06-05,2024-06-03,Herrington Douglas J,CEO Worldwide Amazon Stores,S - Sale,$177.29,"-3,500","529,826",-1%,"-$620,524"
1000,AMZN,2024-07-01,2024-06-27,Zapolsky David,SVP,S - Sale,$195.00,"-4,710","59,500",-7%,"-$918,450"
1001,AMZN,2024-07-03,2024-07-01,Herrington Douglas J,CEO Worldwide Amazon Stores,S - Sale,$199.12,"-16,000","513,828",-3%,"-$3,185,965"
